<p style="text-align:center">
    <a href="https://skills.network/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDS0321ENSkillsNetwork26802033-2022-01-01" target="_blank">
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="200" alt="Skills Network Logo"  />
    </a>
</p>


# **Space X  Falcon 9 First Stage Landing Prediction**


## Web scraping Falcon 9 and Falcon Heavy Launches Records from Wikipedia


Estimated time needed: **40** minutes


In this lab, you will be performing web scraping to collect Falcon 9 historical launch records from a Wikipedia page titled `List of Falcon 9 and Falcon Heavy launches`

[https://en.wikipedia.org/wiki/List_of_Falcon\_9\_and_Falcon_Heavy_launches](https://en.wikipedia.org/wiki/List_of_Falcon\_9\_and_Falcon_Heavy_launches?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDS0321ENSkillsNetwork26802033-2022-01-01)


![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module\_1\_L2/images/Falcon9\_rocket_family.svg)


Falcon 9 first stage will land successfully


![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/api/Images/landing\_1.gif)


Several examples of an unsuccessful landing are shown here:


![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/api/Images/crash.gif)


More specifically, the launch records are stored in a HTML table shown below:


![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module\_1\_L2/images/falcon9-launches-wiki.png)


## Objectives

Web scrap Falcon 9 launch records with `BeautifulSoup`:

*   Extract a Falcon 9 launch records HTML table from Wikipedia
*   Parse the table and convert it into a Pandas data frame


First let's import required packages for this lab


In [1]:
#!pip3 install beautifulsoup4
#!pip3 install requests

In [2]:
import sys

import requests
from bs4 import BeautifulSoup
import re
import unicodedata
import pandas as pd

and we will provide some helper functions for you to process web scraped HTML table


In [3]:
def date_time(table_cells):
    """
    This function returns the data and time from the HTML  table cell
    Input: the  element of a table data cell extracts extra row
    """
    return [data_time.strip() for data_time in list(table_cells.strings)][0:2]

def booster_version(table_cells):
    """
    This function returns the booster version from the HTML  table cell 
    Input: the  element of a table data cell extracts extra row
    """
    out=''.join([booster_version for i,booster_version in enumerate( table_cells.strings) if i%2==0][0:-1])
    return out

def landing_status(table_cells):
    """
    This function returns the landing status from the HTML table cell 
    Input: the  element of a table data cell extracts extra row
    """
    out=[i for i in table_cells.strings][0]
    return out


def get_mass(table_cells):
    mass=unicodedata.normalize("NFKD", table_cells.text).strip()
    if mass:
        mass.find("kg")
        new_mass=mass[0:mass.find("kg")+2]
    else:
        new_mass=0
    return new_mass


def extract_column_from_header(row):
    """
    This function returns the landing status from the HTML table cell 
    Input: the  element of a table data cell extracts extra row
    """
    if (row.br):
        row.br.extract()
    if row.a:
        row.a.extract()
    if row.sup:
        row.sup.extract()
        
    colunm_name = ' '.join(row.contents)
    
    # Filter the digit and empty names
    if not(colunm_name.strip().isdigit()):
        colunm_name = colunm_name.strip()
        return colunm_name    


To keep the lab tasks consistent, you will be asked to scrape the data from a snapshot of the  `List of Falcon 9 and Falcon Heavy launches` Wikipage updated on
`9th June 2021`


In [4]:
static_url = "https://en.wikipedia.org/w/index.php?title=List_of_Falcon_9_and_Falcon_Heavy_launches&oldid=1027686922"

Next, request the HTML page from the above URL and get a `response` object


### TASK 1: Request the Falcon9 Launch Wiki page from its URL


First, let's perform an HTTP GET method to request the Falcon9 Launch HTML page, as an HTTP response.


In [5]:
# use requests.get() method with the provided static_url
# assign the response to a object
response=requests.get(static_url)
#response.text

Create a `BeautifulSoup` object from the HTML `response`


In [6]:
# Use BeautifulSoup() to create a BeautifulSoup object from a response text content (response.text)
soup = BeautifulSoup(response.text,"html.parser")

Print the page title to verify if the `BeautifulSoup` object was created properly


In [7]:
# Use soup.title attribute
tag_object=soup.title
print("tag object:",tag_object)

tag object: <title>List of Falcon 9 and Falcon Heavy launches - Wikipedia</title>


### TASK 2: Extract all column/variable names from the HTML table header


Next, we want to collect all relevant column names from the HTML table header


Let's try to find all tables on the wiki page first. If you need to refresh your memory about `BeautifulSoup`, please check the external reference link towards the end of this lab


In [8]:
# Use the find_all function in the BeautifulSoup object, with element type `table`
# Assign the result to a list called `html_tables`
html_tables=soup.find_all('table')
print('>>> Number of tables in the webpage: ',len(html_tables))

>>> Number of tables in the webpage:  24


Starting from the third table is our target table contains the actual launch records.


In [9]:
# Let's print the third table and check its content
first_launch_table = html_tables[2]
print(first_launch_table)

<table class="wikitable plainrowheaders collapsible" style="width: 100%;">
<tbody><tr>
<th scope="col">Flight No.
</th>
<th scope="col">Date and<br/>time (<a href="/wiki/Coordinated_Universal_Time" title="Coordinated Universal Time">UTC</a>)
</th>
<th scope="col"><a href="/wiki/List_of_Falcon_9_first-stage_boosters" title="List of Falcon 9 first-stage boosters">Version,<br/>Booster</a> <sup class="reference" id="cite_ref-booster_11-0"><a href="#cite_note-booster-11">[b]</a></sup>
</th>
<th scope="col">Launch site
</th>
<th scope="col">Payload<sup class="reference" id="cite_ref-Dragon_12-0"><a href="#cite_note-Dragon-12">[c]</a></sup>
</th>
<th scope="col">Payload mass
</th>
<th scope="col">Orbit
</th>
<th scope="col">Customer
</th>
<th scope="col">Launch<br/>outcome
</th>
<th scope="col"><a href="/wiki/Falcon_9_first-stage_landing_tests" title="Falcon 9 first-stage landing tests">Booster<br/>landing</a>
</th></tr>
<tr>
<th rowspan="2" scope="row" style="text-align:center;">1
</th>
<td>

You should able to see the columns names embedded in the table header elements `<th>` as follows:


```
<tr>
<th scope="col">Flight No.
</th>
<th scope="col">Date and<br/>time (<a href="/wiki/Coordinated_Universal_Time" title="Coordinated Universal Time">UTC</a>)
</th>
<th scope="col"><a href="/wiki/List_of_Falcon_9_first-stage_boosters" title="List of Falcon 9 first-stage boosters">Version,<br/>Booster</a> <sup class="reference" id="cite_ref-booster_11-0"><a href="#cite_note-booster-11">[b]</a></sup>
</th>
<th scope="col">Launch site
</th>
<th scope="col">Payload<sup class="reference" id="cite_ref-Dragon_12-0"><a href="#cite_note-Dragon-12">[c]</a></sup>
</th>
<th scope="col">Payload mass
</th>
<th scope="col">Orbit
</th>
<th scope="col">Customer
</th>
<th scope="col">Launch<br/>outcome
</th>
<th scope="col"><a href="/wiki/Falcon_9_first-stage_landing_tests" title="Falcon 9 first-stage landing tests">Booster<br/>landing</a>
</th></tr>
```


Next, we just need to iterate through the `<th>` elements and apply the provided `extract_column_from_header()` to extract column name one by one


In [10]:
column_names = []

# Apply find_all() function with `th` element on first_launch_table
# Iterate each th element and apply the provided extract_column_from_header() to get a column name
# Append the Non-empty column name (`if name is not None and len(name) > 0`) into a list called column_names
a=first_launch_table.find_all('th')
i=0
for th_el in a:
    name=extract_column_from_header(th_el)
    print(i,': *',name,'*')
    if (name is not None and len(name) > 0):
        column_names.append(name)
        print(i,':',column_names)
    i+=1        

0 : * Flight No. *
0 : ['Flight No.']
1 : * Date and time ( ) *
1 : ['Flight No.', 'Date and time ( )']
2 : *  *
3 : * Launch site *
3 : ['Flight No.', 'Date and time ( )', 'Launch site']
4 : * Payload *
4 : ['Flight No.', 'Date and time ( )', 'Launch site', 'Payload']
5 : * Payload mass *
5 : ['Flight No.', 'Date and time ( )', 'Launch site', 'Payload', 'Payload mass']
6 : * Orbit *
6 : ['Flight No.', 'Date and time ( )', 'Launch site', 'Payload', 'Payload mass', 'Orbit']
7 : * Customer *
7 : ['Flight No.', 'Date and time ( )', 'Launch site', 'Payload', 'Payload mass', 'Orbit', 'Customer']
8 : * Launch outcome *
8 : ['Flight No.', 'Date and time ( )', 'Launch site', 'Payload', 'Payload mass', 'Orbit', 'Customer', 'Launch outcome']
9 : *  *
10 : * None *
11 : * None *
12 : * None *
13 : * None *
14 : * None *
15 : * None *
16 : * None *


Check the extracted column names


In [11]:
print('>>> Column names:')
for i in range(len(column_names)):
    print(i+1,' ',column_names[i])

>>> Column names:
1   Flight No.
2   Date and time ( )
3   Launch site
4   Payload
5   Payload mass
6   Orbit
7   Customer
8   Launch outcome


## TASK 3: Create a data frame by parsing the launch HTML tables


We will create an empty dictionary with keys from the extracted column names in the previous task. Later, this dictionary will be converted into a Pandas dataframe


In [99]:
launch_dict = {}
launch_dict= dict.fromkeys(column_names)

# Remove an irrelevant column
del launch_dict['Date and time ( )']

# Let's initialize the launch_dict with each value to be an empty list
launch_dict['Flight No.'] = []
launch_dict['Launch site'] = []
launch_dict['Payload'] = []
launch_dict['Payload mass'] = []
launch_dict['Orbit'] = []
launch_dict['Customer'] = []
launch_dict['Launch outcome'] = []
# Then, let's add some new columns
launch_dict['Version Booster']=[]
launch_dict['Booster landing']=[]
launch_dict['Date']=[]
launch_dict['Time']=[]

Next, we just need to fill up the `launch_dict` with launch records extracted from table rows.


Usually, HTML tables in Wiki pages are likely to contain unexpected annotations and other types of noises, such as reference links `B0004.1[8]`, missing values `N/A [e]`, inconsistent formatting, etc.


To simplify the parsing process, we have provided an incomplete code snippet below to help you to fill up the `launch_dict`. Please complete the following code snippet with TODOs or you can choose to write your own logic to parse all launch tables:


In [100]:
extracted_row = 0
#Extract each table 
for table_number,table in enumerate(soup.find_all('table',"wikitable plainrowheaders collapsible")):
   # get table row 
    for rows in table.find_all("tr"):
        #check to see if first table heading is as number corresponding to launch a number 
        if rows.th:
            if rows.th.string:
                flight_number=rows.th.string.strip()
                flag=flight_number.isdigit()
                print('=====> flight_number {0} and flag {1}'.format(flight_number,flag))
        else:
            flag=False
        #get table element 
        row=rows.find_all('td')
        #if it is number save cells in a dictonary 
        if flag:
            print('=====> ***{3}***   \n{0}\n, {1}\n, {2}'.format(row[0],row[1],row[6],flight_number))
            
            extracted_row += 1
            # Flight Number value
            # TODO: Append the flight_number into launch_dict with key `Flight No.`
            #print(flight_number)
            # flight_number = extracted_row 
            launch_dict['Flight No.'].append(flight_number)   # MY ADDITION
            datatimelist=date_time(row[0])
            
            # Date value
            # TODO: Append the date into launch_dict with key `Date`
            date = datatimelist[0].strip(',')
            #print(date)
            launch_dict['Date'].append(date)   # MY ADDITION
                        
            # Time value
            # TODO: Append the time into launch_dict with key `Time`
            time = datatimelist[1]
            #print(time)
            launch_dict['Time'].append(time)   # MY ADDITION
              
            # Booster version
            # TODO: Append the bv into launch_dict with key `Version Booster`
            bv=booster_version(row[1])
            if not(bv):
                bv=row[1].a.string
            #print(bv)
            launch_dict['Version Booster'].append(bv)   # MY ADDITION
            
            # Launch Site
            # TODO: Append the bv into launch_dict with key `Launch site`
            launch_site = row[2].a.string
            #print(launch_site)
            launch_dict['Launch site'].append(launch_site)   # MY ADDITION
            
            # Payload
            # TODO: Append the payload into launch_dict with key `Payload`
            payload = row[3].a.string
            #print(payload)
            launch_dict['Payload'].append(payload)   # MY ADDITION
            
            # Payload Mass
            # TODO: Append the payload_mass into launch_dict with key `Payload mass`
            payload_mass = get_mass(row[4])
            #print(payload)
            launch_dict['Payload mass'].append(payload_mass)   # MY ADDITION            
            
            # Orbit
            # TODO: Append the orbit into launch_dict with key `Orbit`
            orbit = row[5].a.string
            #print(orbit)
            launch_dict['Orbit'].append(orbit)   # MY ADDITION
            
            # Customer
            # TODO: Append the customer into launch_dict with key `Customer`
            #customer = row[6].a.string
            customer = row[6].a
            #print(customer)
            launch_dict['Customer'].append(customer)   # MY ADDITION
            
            # Launch outcome
            # TODO: Append the launch_outcome into launch_dict with key `Launch outcome`
            launch_outcome = list(row[7].strings)[0].strip()
            #print(launch_outcome)
            launch_dict['Launch outcome'].append(launch_outcome)   # MY ADDITION
            
            # Booster landing
            # TODO: Append the launch_outcome into launch_dict with key `Booster landing`
            booster_landing = landing_status(row[8])
            #print(booster_landing)
            launch_dict['Booster landing'].append(booster_landing)   # MY ADDITION
            

=====> flight_number Flight No. and flag False
=====> flight_number 1 and flag True
=====> ***1***   
<td>4 June 2010,<br/>18:45
</td>
, <td><a href="/wiki/Falcon_9_v1.0" title="Falcon 9 v1.0">F9 v1.0</a><sup class="reference" id="cite_ref-MuskMay2012_13-0"><a href="#cite_note-MuskMay2012-13">[7]</a></sup><br/>B0003.1<sup class="reference" id="cite_ref-block_numbers_14-0"><a href="#cite_note-block_numbers-14">[8]</a></sup>
</td>
, <td><a href="/wiki/SpaceX" title="SpaceX">SpaceX</a>
</td>
=====> flight_number 2 and flag True
=====> ***2***   
<td>8 December 2010,<br/>15:43<sup class="reference" id="cite_ref-spaceflightnow_Clark_Launch_Report_19-0"><a href="#cite_note-spaceflightnow_Clark_Launch_Report-19">[13]</a></sup>
</td>
, <td><a href="/wiki/Falcon_9_v1.0" title="Falcon 9 v1.0">F9 v1.0</a><sup class="reference" id="cite_ref-MuskMay2012_13-1"><a href="#cite_note-MuskMay2012-13">[7]</a></sup><br/>B0004.1<sup class="reference" id="cite_ref-block_numbers_14-1"><a href="#cite_note-bloc

In [101]:
# Checking the contents of launch_dict
for i in launch_dict.keys():
    print(i,': ',launch_dict[i][105:107])
    

Flight No. :  ['106', '107']
Launch site :  ['CCSFS', 'CCSFS']
Payload :  ['Transporter-1', 'Starlink']
Payload mass :  ['~5,000 kg', '15,600 kg']
Orbit :  ['SSO', 'LEO']
Customer :  [None, <a href="/wiki/SpaceX" title="SpaceX">SpaceX</a>]
Launch outcome :  ['Success', 'Success']
Version Booster :  ['F9 B5B1058.5', 'F9 B5 ♺']
Booster landing :  ['Success', 'Success']
Date :  ['24 January 2021', '4 February 2021']
Time :  ['15:00', '06:19']


In [102]:
print('Length of the elements in the launch_dict dictionary:')
print('Flight No. .....: ',len(launch_dict['Flight No.']))
print('Launch site ....: ',len(launch_dict['Launch site']))
print('Payload ........: ',len(launch_dict['Payload']))
print('Payload mass ...: ',len(launch_dict['Payload mass']))
print('Orbit ..........: ',len(launch_dict['Orbit']))
print('Customer .......: ',len(launch_dict['Customer']))
print('Launch outcome .: ',len(launch_dict['Launch outcome']))
print('Version Booster : ',len(launch_dict['Version Booster']))
print('Booster landing : ',len(launch_dict['Booster landing']))
print('Date ...........: ',len(launch_dict['Date']))
print('Time ...........: ',len(launch_dict['Time']))  

Length of the elements in the launch_dict dictionary:
Flight No. .....:  121
Launch site ....:  121
Payload ........:  121
Payload mass ...:  121
Orbit ..........:  121
Customer .......:  121
Launch outcome .:  121
Version Booster :  121
Booster landing :  121
Date ...........:  121
Time ...........:  121


After you have fill in the parsed launch record values into `launch_dict`, you can create a dataframe from it.


In [121]:
df=pd.DataFrame(launch_dict)

In [122]:
df.describe()

,Flight No.,Launch site,Payload,Payload mass,Orbit,Customer,Launch outcome,Version Booster,Booster landing,Date,Time
count,121,121,121,121,121,120,121,121,121,121,121
unique,121,5,82,84,8,41,2,52,8,121,116
top,1,CCAFS,Starlink,"15,600 kg",LEO,[NASA],Success,F9 B5,Success,4 June 2010,04:45
freq,1,40,29,24,67,32,120,26,80,1,3


In [123]:
print('df.dtypes:\n',df.dtypes,'\n\n df.shape: ',df.shape,'\n\n df.isnull().sum():\n',df.isnull().sum()) 

df.dtypes:
 Flight No.         object
Launch site        object
Payload            object
Payload mass       object
Orbit              object
Customer           object
Launch outcome     object
Version Booster    object
Booster landing    object
Date               object
Time               object
dtype: object 

 df.shape:  (121, 11) 

 df.isnull().sum():
 Flight No.         0
Launch site        0
Payload            0
Payload mass       0
Orbit              0
Customer           1
Launch outcome     0
Version Booster    0
Booster landing    0
Date               0
Time               0
dtype: int64


In [124]:
# Somehow, one customer is set to 'None'. In reality, the value for it is 'Various'
df_aux = df[df['Customer'].isnull()]
print(df_aux)
df['Customer'][100:]

    Flight No. Launch site        Payload Payload mass Orbit Customer  \
105        106       CCSFS  Transporter-1    ~5,000 kg   SSO     None   

    Launch outcome Version Booster Booster landing             Date   Time  
105        Success    F9 B5B1058.5         Success  24 January 2021  15:00  


100         [NASA]
101    [Sirius XM]
102          [NRO]
103      [Türksat]
104       [SpaceX]
105           None
106       [SpaceX]
107       [SpaceX]
108       [SpaceX]
109       [SpaceX]
110       [SpaceX]
111       [SpaceX]
112       [SpaceX]
113         [NASA]
114       [SpaceX]
115       [SpaceX]
116       [SpaceX]
117       [SpaceX]
118       [SpaceX]
119         [NASA]
120    [Sirius XM]
Name: Customer, dtype: object

In [125]:
# Fixing the issue reported above
df['Customer'] = df['Customer'].fillna('Various')
#df['Customer']=df['Customer'].astype(str)
print('df.dtypes:\n',df.dtypes)
print(df['Customer'][100:])

df.dtypes:
 Flight No.         object
Launch site        object
Payload            object
Payload mass       object
Orbit              object
Customer           object
Launch outcome     object
Version Booster    object
Booster landing    object
Date               object
Time               object
dtype: object
100         [NASA]
101    [Sirius XM]
102          [NRO]
103      [Türksat]
104       [SpaceX]
105        Various
106       [SpaceX]
107       [SpaceX]
108       [SpaceX]
109       [SpaceX]
110       [SpaceX]
111       [SpaceX]
112       [SpaceX]
113         [NASA]
114       [SpaceX]
115       [SpaceX]
116       [SpaceX]
117       [SpaceX]
118       [SpaceX]
119         [NASA]
120    [Sirius XM]
Name: Customer, dtype: object


In [126]:
df.tail(30)

,Flight No.,Launch site,Payload,Payload mass,Orbit,Customer,Launch outcome,Version Booster,Booster landing,Date,Time
91,92,CCAFS,SAOCOM 1B,"3,130 kg",SSO,[CONAE],Success,F9 B5,Success,30 August 2020,23:18
92,93,KSC,Starlink,"15,600 kg",LEO,[SpaceX],Success,F9 B5B1060.2,Success,3 September 2020,12:46:14
93,94,KSC,Starlink,"15,600 kg",LEO,[SpaceX],Success,F9 B5B1058.3,Success,6 October 2020,11:29:34
94,95,KSC,Starlink,"15,600 kg",LEO,[SpaceX],Success,F9 B5B1051.6,Success,18 October 2020,12:25:57
95,96,CCAFS,Starlink,"15,600 kg",LEO,[SpaceX],Success,F9 B5,Success,24 October 2020,15:31:34
96,97,CCAFS,GPS III,"4,311 kg",MEO,[USSF],Success,F9 B5,Success,5 November 2020,23:24:23
97,98,KSC,Crew-1,"~12,500 kg",LEO,[NASA],Success,F9 B5,Success,16 November 2020,00:27
98,99,VAFB,Sentinel-6 Michael Freilich (Jason-CS A),"1,192 kg",LEO,[NASA],Success,F9 B5,Success,21 November 2020,17:17:08
99,100,CCAFS,Starlink,"15,600 kg",LEO,[SpaceX],Success,F9 B5 ♺,Success,25 November 2020,02:13
100,101,KSC,SpaceX CRS-21,"2,972 kg",LEO,[NASA],Success,F9 B5 ♺,Success,6 December 2020,16:17:08


We can now export it to a <b>CSV</b> for the next section, but to make the answers consistent and in case you have difficulties finishing this lab.

Following labs will be using a provided dataset to make each lab independent.


<code>df.to_csv('spacex_web_scraped.csv', index=False)</code>


## Authors


<a href="https://www.linkedin.com/in/yan-luo-96288783/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDS0321ENSkillsNetwork26802033-2022-01-01">Yan Luo</a>


<a href="https://www.linkedin.com/in/nayefaboutayoun/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDS0321ENSkillsNetwork26802033-2022-01-01">Nayef Abou Tayoun</a>


## Change Log


| Date (YYYY-MM-DD) | Version | Changed By | Change Description          |
| ----------------- | ------- | ---------- | --------------------------- |
| 2021-06-09        | 1.0     | Yan Luo    | Tasks updates               |
| 2020-11-10        | 1.0     | Nayef      | Created the initial version |


Copyright © 2021 IBM Corporation. All rights reserved.
